In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import AdaBoostClassifier

In [2]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
y = data.target

In [3]:
clf = AdaBoostClassifier().fit(X, y)
scoring = {'accuracy': 'accuracy', 'f1-score': 'f1_macro', 'recall': 'recall', 'precision': 'precision'}
score = cross_validate(clf, X, y, return_train_score=True, scoring=scoring)

c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\sklearn\model_selection\_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\sklearn\metrics\_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\sklearn\utils\_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\sklearn

In [15]:
for i, j in score.items():
    if i == 'fit_time' or i == 'score_time':
        continue
    else:
        print(i, j)

test_accuracy [0.96666667 0.93333333 0.9        0.93333333 1.        ]
train_accuracy [0.95       0.925      0.975      0.96666667 0.95      ]
test_f1-score [0.96658312 0.93265993 0.89974937 0.93265993 1.        ]
train_f1-score [0.94996873 0.92403861 0.97496479 0.96666667 0.94996873]
test_recall [nan nan nan nan nan]
train_recall [nan nan nan nan nan]
test_precision [nan nan nan nan nan]
train_precision [nan nan nan nan nan]


In [4]:
score

{'fit_time': array([0.06499839, 0.06600046, 0.06702209, 0.06401134, 0.06600237]),
 'score_time': array([0.01500082, 0.01199937, 0.01198936, 0.01001143, 0.01099706]),
 'test_accuracy': array([0.96666667, 0.93333333, 0.9       , 0.93333333, 1.        ]),
 'train_accuracy': array([0.95      , 0.925     , 0.975     , 0.96666667, 0.95      ]),
 'test_f1-score': array([0.96658312, 0.93265993, 0.89974937, 0.93265993, 1.        ]),
 'train_f1-score': array([0.94996873, 0.92403861, 0.97496479, 0.96666667, 0.94996873]),
 'test_recall': array([nan, nan, nan, nan, nan]),
 'train_recall': array([nan, nan, nan, nan, nan]),
 'test_precision': array([nan, nan, nan, nan, nan]),
 'train_precision': array([nan, nan, nan, nan, nan])}

In [5]:
import bentoml, mlflow, optuna, uuid
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

UNIQUE_PREFIX = str(uuid.uuid4())[:8]
def objective(trial, X, y):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
    }
    mlflow.log_params(param)
    model = AdaBoostClassifier(**param)
    scoring = {'accuracy': 'accuracy', 'recall': 'recall', 'precision': 'precision', 'f1_weighted': 'f1_weighted'}
    scores = cross_validate(model, X, y, cv=5, scoring=scoring)

    mlflow.log_metrics({'scores', scores})
    return scores['train_score'].mean()

def ada_clf_run(X, y):
    mlflow.set_tracking_uri('http://127.0.0.1:8081')
    study = optuna.create_study(study_name='ada-hpo', direction='maximize', load_if_exists=True)
    study.optimize(lambda trial: objective(trial, X, y), n_trials=5)
    best_param = study.best_params
    with mlflow.start_run(run_name='Best_model'):
        mlflow.log_params(best_param)
        best_model = AdaBoostClassifier(**best_param).fit(X, y)
        scoring = {'accuracy': 'accuracy', 'recall': 'recall', 'precision': 'precision', 'f1_weighted': 'f1_weighted'}
        scores = cross_validate(best_model, X, y, cv=5, scoring=scoring)

        mlflow.log_metrics({'scores', scores})
        model_name = f'ada_clf_model_{UNIQUE_PREFIX}'

        mlflow.sklearn.log_model(best_model, model_name)
        bentoml.mlflow.import_model(
                                    'Best_model',
                                    model_uri="runs:/<mlflow_run_id>/run-relative/path/to/model",
                                    signatures={
                                                "predict": {"batchable": True},
                                                }
                                    )
        run_id = mlflow.active_run().info.run_id

        # Construct the model URI
        model_uri = f"runs:/{run_id}/run-relative/path/to/model"

        # BentoML import_model
        bentoml.mlflow.import_model(
            'Best_model',
            model_uri=model_uri,
            signatures={"predict": {"batchable": True}}
        )

c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\pydantic\_internal\_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\pydantic\_internal\_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
for i, j in score.items():
    if i == 'fit_time' or i == 'score_time':
        continue
    else:
        print(i, j)

test_accuracy [0.96666667 0.93333333 0.9        0.93333333 1.        ]
train_accuracy [0.95       0.925      0.975      0.96666667 0.95      ]
test_f1-score [0.96658312 0.93265993 0.89974937 0.93265993 1.        ]
train_f1-score [0.94996873 0.92403861 0.97496479 0.96666667 0.94996873]
test_recall [nan nan nan nan nan]
train_recall [nan nan nan nan nan]
test_precision [nan nan nan nan nan]
train_precision [nan nan nan nan nan]


In [17]:
import bentoml, mlflow, optuna, uuid
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

UNIQUE_PREFIX = str(uuid.uuid4())[:8]
def objective(trial, X, y):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
    }
    mlflow.log_params(param)
    model = AdaBoostClassifier(**param)
    scoring = ['accuracy', 'recall', 'precision', 'f1_weighted']
    scores = cross_validate(model, X, y, cv=5, scoring=scoring)

    # mlflow.log_metrics('scores', scores)
    # mlflow.log_metric('train_accuracy', scores['train_accuracy'].mean())

    # mlflow.log_metric('test_accuracy', scores['test_accuracy'].mean())
    # mlflow.log_metric('test_recall', scores['test_recall'].mean())
    # mlflow.log_metric('test_precision', scores['test_precision'].mean())
    # mlflow.log_metric('test_f1_weighted', scores['test_f1_weighted'].mean())
    for metric_name, score_value in scores.items():
        if metric_name == 'fit_time' or metric_name == 'score_time':
            continue
        else:
            mlflow.log_metric(metric_name, score_value.mean())
            
    return scores['train_score'].mean()
mlflow.set_tracking_uri('http://127.0.0.1:8081')
study = optuna.create_study(study_name='ada-hpo', direction='maximize', load_if_exists=True)
study.optimize(lambda trial: objective(trial, X, y), n_trials=5)
best_param = study.best_params

[I 2023-11-24 00:16:32,543] A new study created in memory with name: ada-hpo
[W 2023-11-24 00:17:32,594] Trial 0 failed with parameters: {'n_estimators': 97, 'learning_rate': 0.29069278361062917} because of the following error: MlflowException("API request to http://127.0.0.1:8081/api/2.0/mlflow/runs/log-batch failed with exception HTTPConnectionPool(host='127.0.0.1', port=8081): Max retries exceeded with url: /api/2.0/mlflow/runs/log-batch (Caused by ResponseError('too many 500 error responses'))").
urllib3.exceptions.ResponseError: too many 500 error responses

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\requests\adapters.py", line 486, in send
    resp = conn.urlopen(
           ^^^^^^^^^^^^^
  File "c:\Users\jw160\OneDrive\test\bento2\.venv_bento2\Lib\site-packages\urllib3\connectionpool.py", line 946, in urlopen
    return self.urlopen(
          

MlflowException: API request to http://127.0.0.1:8081/api/2.0/mlflow/runs/log-batch failed with exception HTTPConnectionPool(host='127.0.0.1', port=8081): Max retries exceeded with url: /api/2.0/mlflow/runs/log-batch (Caused by ResponseError('too many 500 error responses'))